<a href="https://colab.research.google.com/github/fajar140/Project-ML-PrediksiHargaMobilBekas/blob/main/ProjectAslab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Prediksi Harga Mobil Bekas Menggunakan Metode Random Forest Regression

**Nama:Fajar Ibrahim Nasry Hamdan**

**Npm: 51421685**

**Kelas: 3IA02**

Dalam analisis harga mobil bekas, prediksi harga merupakan tantangan yang kompleks karena banyaknya faktor yang mempengaruhi nilai jual sebuah kendaraan. Untuk itu, penggunaan model machine learning seperti Random Forest Regression dapat memberikan wawasan yang mendalam dan akurat tentang harga mobil bekas berdasarkan berbagai fitur yang relevan.

Model prediksi harga mobil bekas diterapkan menggunakan metode Random Forest Regression. Dataset yang digunakan untuk membangun model ini diperoleh dari Kaggle, sebuah platform kompetisi data terkemuka yang menyediakan berbagai dataset untuk analisis dan pengembangan model machine learning.

Dengan menggunakan dataset ini, akan memanfaatkan algoritma Random Forest Regression untuk memodelkan hubungan antara fitur-fitur mobil dan harga jualnya. Random Forest adalah metode ensemble yang menggabungkan hasil dari beberapa pohon keputusan untuk meningkatkan akurasi prediksi dan mengurangi overfitting. Melalui pendekatan ini, diharapkan model yang dihasilkan dapat memberikan estimasi harga yang lebih akurat dan bermanfaat bagi konsumen maupun penjual dalam pasar mobil bekas.

Berikut adalah link dataset yang digunakan:
https://www.kaggle.com/datasets/pushpakhinglaspure/used-car-price-prediction

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

### Load Data

In [2]:
data = pd.read_csv('cardekho_data.csv')

### Check dataset

In [3]:
#Cek Dimensi Dataset
data.shape

(301, 9)

In [4]:
# Menampilkan beberapa baris pertama
data.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [5]:
# Cek Kelengkapan Dataset
data.isna().sum()

,0
Car_Name,0
Year,0
Selling_Price,0
Present_Price,0
Kms_Driven,0
Fuel_Type,0
Seller_Type,0
Transmission,0
Owner,0


In [6]:
# Cek Tipe data
data.dtypes

,0
Car_Name,object
Year,int64
Selling_Price,float64
Present_Price,float64
Kms_Driven,int64
Fuel_Type,object
Seller_Type,object
Transmission,object
Owner,int64


In [7]:
# Melihat jumlah nilai unik di kolom 'Fuel_Type'
print(data['Fuel_Type'].value_counts())

# Melihat jumlah nilai unik di kolom 'Seller_Type'
print(data['Seller_Type'].value_counts())

# Melihat jumlah nilai unik di kolom 'Transmission'
print(data['Transmission'].value_counts())


Fuel_Type
Petrol    239
Diesel     60
CNG         2
Name: count, dtype: int64
Seller_Type
Dealer        195
Individual    106
Name: count, dtype: int64
Transmission
Manual       261
Automatic     40
Name: count, dtype: int64


In [8]:
# Mengganti nilai unik pada kolom 'Fuel_Type'
data['Fuel_Type'] = data['Fuel_Type'].map({'Petrol': 1, 'Diesel': 2, 'CNG': 3})

# Mengganti nilai unik pada kolom 'Seller_Type'
data['Seller_Type'] = data['Seller_Type'].map({'Dealer': 1, 'Individual': 2})

# Mengganti nilai unik pada kolom 'Transmission'
data['Transmission'] = data['Transmission'].map({'Manual': 1, 'Automatic': 2})

# Menampilkan hasil setelah perubahan
print(data['Fuel_Type'].value_counts())
print(data['Seller_Type'].value_counts())
print(data['Transmission'].value_counts())

Fuel_Type
1    239
2     60
3      2
Name: count, dtype: int64
Seller_Type
1    195
2    106
Name: count, dtype: int64
Transmission
1    261
2     40
Name: count, dtype: int64


### Deleting Unused Columns

In [9]:
# Menghapus Kolom Yang Tidak Digunakan
data.drop(columns = ['Car_Name'], inplace=True)

### Identifying and Handling Outliers

In [10]:
outliers_percentages = []

for col in data.columns:
    q1 = data[col].quantile(0.25)
    q3 = data[col].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers_count = ((data[col] < lower_bound) | (data[col] > upper_bound)).sum()
    outliers_percentage = (outliers_count / len(data[col])) * 100

    outliers_percentages.append({'Column': col, 'Outlier Percentage': outliers_percentage})

outliers_data = pd.DataFrame(outliers_percentages)
outliers_data = outliers_data.sort_values(by='Outlier Percentage', ascending=False)

In [11]:
for col in data.columns:
    q1 = data[col].quantile(0.25)
    q3 = data[col].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # Calculate outliers
    outliers = (data[col] < lower_bound) | (data[col] > upper_bound)
    outlier_percentage = outliers.mean() * 100


    data[col] = data[col].where(~outliers, np.mean(data[col]))

In [12]:
data.describe()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
count,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000
mean,2013.845167,3.764691,6.363949,32766.277900,1.043796,1.352159,1.017660,0.001578
std,2.501835,2.970700,5.071754,21159.217131,0.086132,0.478439,0.045186,0.008118
min,2006.000000,0.100000,0.320000,500.000000,1.000000,1.000000,1.000000,0.000000
25%,2012.000000,0.900000,1.200000,15000.000000,1.000000,1.000000,1.000000,0.000000
50%,2014.000000,3.600000,6.400000,32000.000000,1.000000,1.000000,1.000000,0.000000
75%,2016.000000,5.400000,9.400000,45078.000000,1.000000,2.000000,1.000000,0.000000
max,2018.000000,12.900000,22.830000,92233.000000,1.212625,2.000000,1.132890,0.043189


### Random Forest Regression Model

In [13]:
x = data.drop('Selling_Price', axis=1)
y = data['Selling_Price']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.8, random_state = 42)

In [15]:
# check the shape of X_train and X_test
x_train.shape, x_test.shape

((60, 7), (241, 7))

### Model Initialization and Training

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [17]:
rf_model = RandomForestRegressor(n_estimators= 49, criterion="squared_error", max_depth= 67)

In [18]:
rf_model.fit(x_train, y_train)

RandomForestRegressor(max_depth=67, n_estimators=49)

### Model Prediction and Evaluation

In [19]:
y_pred = rf_model.predict(x_test)

In [20]:
# Calculate Mean Absolute Error (MAE) as a metric of model performance
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.7436308671605556


In [21]:
# Calculate Root Mean Squared Error (RMSE) as a metric of model performance
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 1.2566493364474423


### Save Model

In [22]:
import joblib

# Save the model
joblib.dump(rf_model, 'trained_rf_model.pkl')

['trained_rf_model.pkl']

### Test Model

In [23]:
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Import ipywidgets dan display
import ipywidgets as widgets
from IPython.display import display

In [24]:
# Load the model
model = joblib.load('trained_rf_model.pkl')

In [25]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# Definisikan widget input untuk setiap pertanyaan
Year = widgets.FloatText(
    value=2020,
    description='Tahun Produksi:',
    disabled=False
)

Present_Price = widgets.FloatText(
    value=0.0,
    description='Harga Beli Mobil:',
    disabled=False
)

Kms_Driven = widgets.FloatText(
    value=0,
    description='Kilometer Ditempuh:',
    disabled=False
)

Fuel_Type = widgets.RadioButtons(
    options=[('Petrol', 1), ('Diesel', 2), ('CNG', 3)],
    description='Tipe Bahan Bakar:',
    disabled=False
)

Seller_Type = widgets.RadioButtons(
    options=[('Dealer', 1), ('Individual', 2)],
    description='Tipe Penjualan:',
    disabled=False
)

Transmission = widgets.RadioButtons(
    options=[('Manual', 1), ('Automatic', 2)],
    description='Tipe Transmisi:',
    disabled=False
)

Owner = widgets.FloatText(
    value=0,
    description='Jumlah Owner Sebelumnya:',
    disabled=False
)

# Tombol submit
submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger'
    tooltip='Klik untuk Submit'
)

# Menampilkan semua widget
display(Year, Present_Price, Kms_Driven, Fuel_Type, Seller_Type, Transmission, Owner, submit_button)

# Fungsi untuk menampilkan prediksi saat tombol di-submit
def on_submit_button_clicked(b):
    # Mengambil nilai dari widget dan membuat DataFrame
    input_data = pd.DataFrame({
        'Year': [Year.value],
        'Present_Price': [Present_Price.value],
        'Kms_Driven': [Kms_Driven.value],
        'Fuel_Type': [Fuel_Type.value],
        'Seller_Type': [Seller_Type.value],
        'Transmission': [Transmission.value],
        'Owner' : [Owner.value]  # Ambil nilai (.value) dari FloatText Owner
    })

    # Melakukan prediksi dengan model
    predicted_price = model.predict(input_data)

    # Menampilkan hasil prediksi
    print(f'Prediksi Harga Jual Mobil Adalah: {predicted_price[0]} USD')

# Menghubungkan tombol submit dengan fungsi prediksi
submit_button.on_click(on_submit_button_clicked)


FloatText(value=2020.0, description='Tahun Produksi:')

FloatText(value=0.0, description='Harga Beli Mobil:')

FloatText(value=0.0, description='Kilometer Ditempuh:')

RadioButtons(description='Tipe Bahan Bakar:', options=(('Petrol', 1), ('Diesel', 2), ('CNG', 3)), value=1)

RadioButtons(description='Tipe Penjualan:', options=(('Dealer', 1), ('Individual', 2)), value=1)

RadioButtons(description='Tipe Transmisi:', options=(('Manual', 1), ('Automatic', 2)), value=1)

FloatText(value=0.0, description='Jumlah Owner Sebelumnya:')

Button(button_style='success', description='Submit', style=ButtonStyle(), tooltip='Klik untuk Submit')

Prediksi Harga Jual Mobil Adalah: 4.898189707776798 USD


### Hasil Dan Kesimpulan

Model prediksi harga mobil bekas yang telah dikembangkan berhasil dan dapat digunakan untuk menghasilkan prediksi dengan tingkat akurasi yang baik. Model ini menunjukkan performa yang unggul dengan nilai Mean Absolute Error (MAE) sebesar 0.7436 dan Root Mean Squared Error (RMSE) sebesar 1.256. Nilai ini lebih rendah dibandingkan dengan hasil penelitian oleh Bambang Kriswantara dan Rifki Sadikin pada tahun 2022, di mana model Random Forest yang mereka uji memiliki MAE sebesar 1.006 dan RMSE sebesar 1.452.

Kesimpulannya, model Random Forest yang digunakan dalam studi ini menunjukkan kinerja yang lebih baik dalam memprediksi harga mobil bekas, menghasilkan error yang lebih rendah dibandingkan penelitian sebelumnya. Ini menunjukkan bahwa model yang stabil dan tepat dalam machine learning dapat memberikan prediksi harga mobil bekas secara akurat dan efektif.

### Referesi

1. Kriswantara, B. & Sadikin, R., 2022. Used Car Price Prediction with Random
Forest Regressor Model. Journal of Information Systems, Informatics and
Computing, 6(1), pp. 40-49.
2. https://www.kaggle.com/datasets/pushpakhinglaspure/used-car-price-prediction